In [19]:
import carla
import numpy as np
import cv2
import time
import sys
import random
from ultralytics import YOLO
from matplotlib import cm
sys.path.append('F:\CARLA\Windows\CARLA_0.9.15\PythonAPI\carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner
import open3d as o3d
from datetime import datetime
import math

In [20]:
client = carla.Client('localhost', 2000)
world = client.get_world()
blueprints = [bp for bp in world.get_blueprint_library().filter('*')]

In [21]:
spawn_points = world.get_map().get_spawn_points()
vehicle_bp = world.get_blueprint_library().filter('*mini*')

vehicle_start_point = spawn_points[1]
vehicle = world.try_spawn_actor(vehicle_bp[0], vehicle_start_point)

numberofSpawnPoints = len(spawn_points)
numberofbikers = 30
random.shuffle(spawn_points)

traffic_manager = client.get_trafficmanager()
traffic_manager.set_global_distance_to_leading_vehicle(2.5)


SetAutopilot = carla.command.SetAutopilot
SpawnActor = carla.command.SpawnActor
FutureActor = carla.command.FutureActor

bikerlist = []

for i in range(numberofbikers):

    bicycle_bp = world.get_blueprint_library().filter('*crossbike*')[0]
    bicycle_bp.set_attribute('role_name', 'autopilot')
    transform = spawn_points[i % numberofSpawnPoints]
    spawn_command = SpawnActor(bicycle_bp, transform).then(SetAutopilot(FutureActor, True))
    bikerlist.append(spawn_command)

client.apply_batch_sync(bikerlist, True)





# Setup base camera

In [22]:
#setup the camera
CAMERA_POS_Z = 1.5 
CAMERA_POS_X = 0 

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')
camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))
vehicle.set_autopilot(True)
model = YOLO("best.pt")



### Bicycle Detection

### Depth Camera

In [23]:
depth_cam_bp = world.get_blueprint_library().find('sensor.camera.depth')
depth_cam_bp.set_attribute('image_size_x', '640')
depth_cam_bp.set_attribute('image_size_y', '360')
depth_cam_bp.set_attribute('fov', '90')
depth_cam_location = carla.Transform(carla.Location(x=CAMERA_POS_X, z=CAMERA_POS_Z))  # Offset from vehicle
depth_camera = world.spawn_actor(depth_cam_bp, depth_cam_location, attach_to=vehicle)

In [24]:
image_w = depth_cam_bp.get_attribute('image_size_x').as_int()
image_h = depth_cam_bp.get_attribute('image_size_y').as_int()

depth_data = {'image': np.zeros((image_h,image_w,4))}
def depth_callback(image,data_dict):
    image.convert(carla.ColorConverter.LogarithmicDepth)
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))
    #depth_data = np.frombuffer(image.raw_data, dtype=np.uint8).reshape((image.height, image.width, 4))

depth_camera.listen(lambda image: depth_callback(image,depth_data))


### SHOW all camera together

In [ ]:
def decode_depth_image(depth_image):
    # Extract depth channels and decode to meters
    depth_image = depth_image[:, :, :3].astype(np.float32)
    depth_image = (depth_image[:, :, 2] + depth_image[:, :, 1] * 256 + depth_image[:, :, 0] * 256*256) / (256*256*256 - 1)
    depth_image *= 1000  # Convert to meters

    return depth_image 


#Bounding boxes distances
def get_object_distance(depthimage, bbox):
    x1, y1, x2, y2 = bbox

    depthimage = decode_depth_image(depthimage)

    y1, y2 = max(0, int(y1)), min(depthimage.shape[0], int(y2))
    x1, x2 = max(0, int(x1)), min(depthimage.shape[1], int(x2))

    # Extract the object depth within the bounding box
    object_depth = depthimage[y1:y2, x1:x2]
    print(object_depth)
    height, width = object_depth.shape

    # Calculate the center indices
    center_y = height // 2
    center_x = width // 2

    # Prepare list for middle points
    middle_points = []

    # Calculate the four middle points based on the center
    if height > 1 and width > 1:  # Ensure there's enough data to get middle points
        middle_points = [
            (center_x - 1, center_y - 1),  # Top-left
            (center_x - 1, center_y),      # Middle-left
            (center_x, center_y - 1),      # Top-right
            (center_x, center_y)           # Middle-right
        ]
    
    valid_points = [(x, y) for x, y in middle_points if 0 <= x < width and 0 <= y < height]
    depth_values = [object_depth[y, x] for x, y in valid_points]
    print(depth_values)

    if object_depth.size > 0:
        distance = np.mean(depth_values)  # Use np.min(object_depth) for closest point
        return distance
    else:
        return None


In [ ]:
while True:
    # Decode depth image for visualization
    #depth_frame = decode_depth_image(depth_data['image'])
    depth_frame = depth_data['image']
    print(depth_frame.shape)
    frame = camera_data['image']
    # Convert RGB image from BGRA to BGR
    frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)
    
    # Run the object detection model on the RGB frame
    results = model(frame)
    for result in results:
        for box in result.boxes:
            # Extract box coordinates and other details
            x1, y1, x2, y2 = box.xyxy[0]
              # Bounding box coordinates
            conf = box.conf[0]            # Confidence score
            cls = box.cls[0]              # Class label
            bicycle_distance = get_object_distance(depth_frame, box.xyxy[0])
            # Draw the bounding box and label on the frame
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.rectangle(depth_frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            label = f"{model.names[int(cls)]}: {conf:.2f}"
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            try:
                distance = f"Distance to bicycle: {bicycle_distance:.2f} meters"
                cv2.putText(frame, distance, (int(x1), int(y1)-30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            except:
                print(bicycle_distance)
    # Combine RGB and depth images side by side
    #combined_image = np.hstack((frame, depth_frame))
    
    # Display the combined frame
    cv2.imshow('Fake self-driving', depth_frame)
    cv2.imshow('RGB Camera', frame)

    # Exit loop on 'q' key press
    if cv2.waitKey(1) == ord('q'):
        break


# Cleanup
cv2.destroyAllWindows()
#depth_camera.stop()

In [28]:
#Destroying the existing things
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()